In [ ]:
import tensorflow as tf
import pandas as pd
import joblib
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

In [8]:
# 파일 경로 설정
file_path = ".\통합데이터_전처리_당뇨고혈압추가.csv"
health_data = pd.read_csv(file_path)

C:\Users\Master\AppData\Local\Temp\ipykernel_6348\113614494.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  health_data = pd.read_csv(file_path)


#고혈압

In [9]:
# 선택할 특성과 타겟 변수
selected_features = ['성별', '연령대코드', '신장', '체중', '수축기 혈압', '이완기 혈압']
health_data['고혈압 여부'] = health_data['고혈압 단계'].apply(lambda x: 1 if x in [1, 2] else 0)
target = '고혈압 여부'

# 데이터프레임에서 필요한 컬럼만 선택
df_selected = health_data[selected_features + [target]]

# 데이터 확인
print(df_selected.head())
print(df_selected.info())


   성별  연령대코드   신장   체중  수축기 혈압  이완기 혈압  고혈압 여부
0   1     13  160   70   109.0    73.0       0
1   1      8  170  110   139.0    87.0       1
2   1      6  175   70   120.0    72.0       1
3   1     11  170   75   140.0    80.0       1
4   2     11  150   65   123.0    69.0       1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630590 entries, 0 to 2630589
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   성별      int64  
 1   연령대코드   int64  
 2   신장      int64  
 3   체중      int64  
 4   수축기 혈압  float64
 5   이완기 혈압  float64
 6   고혈압 여부  int64  
dtypes: float64(2), int64(5)
memory usage: 140.5 MB
None


In [10]:
X = df_selected[selected_features]
y = df_selected[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"훈련 세트 크기: {X_train.shape}")
print(f"테스트 세트 크기: {X_test.shape}")


훈련 세트 크기: (2104472, 6)
테스트 세트 크기: (526118, 6)


In [11]:
results = {}

In [12]:
# 1. Logistic Regression
print("Logistic Regression 학습 중...")
logistic_model = LogisticRegression(random_state=42, max_iter=1000)

# 모델 학습
logistic_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred_logistic = logistic_model.predict(X_test)

# 성능 저장
results['LogisticRegression'] = {
    "accuracy": accuracy_score(y_test, y_pred_logistic),
    "classification_report": classification_report(y_test, y_pred_logistic)
}

# 결과 출력
print("Logistic Regression 결과:")
print("Accuracy:", results['LogisticRegression']['accuracy'])
print("Classification Report:")
print(results['LogisticRegression']['classification_report'])

Logistic Regression 학습 중...
Logistic Regression 결과:
Accuracy: 0.9406311892009017
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92    202771
           1       0.95      0.96      0.95    323347

    accuracy                           0.94    526118
   macro avg       0.94      0.94      0.94    526118
weighted avg       0.94      0.94      0.94    526118



In [13]:
# 모델 저장
model_save_path = "hypertension_model.joblib"
joblib.dump(logistic_model, model_save_path)
print(f"모델이 '{model_save_path}'에 저장되었습니다.")

모델이 'hypertension_model.joblib'에 저장되었습니다.


In [ ]:
# 1. 모델 로드
def load_model(file_path):
    """
    Load a model saved as a .joblib file.
    """
    return joblib.load(file_path)

# 2. 예측 함수
def predict_disease_for_one(model, input_data):

    # 입력 데이터를 DataFrame으로 변환
    input_df = pd.DataFrame([input_data])

    # 입력 데이터 검증
    required_columns = ['성별', '연령대코드', '신장', '체중', '수축기 혈압', '이완기 혈압']
    if not all(col in input_df.columns for col in required_columns):
        raise ValueError(f"Input data must contain columns: {required_columns}")

    # 추론 시작 시간
    start_time = time.time()

    # 모델 예측 (확률값)
    probabilities = model.predict_proba(input_df)

    # 추론 종료 시간
    inference_time = time.time() - start_time

    # 가장 높은 확률과 해당 클래스 추출
    max_prob_idx = probabilities.argmax(axis=1)[0]  # 확률이 가장 높은 클래스 인덱스
    max_prob = probabilities[0][max_prob_idx]       # 해당 확률
    max_class = model.classes_[max_prob_idx]        # 해당 클래스

    return max_class, max_prob, inference_time

'''def get_user_input():
    """
    사용자로부터 입력 데이터를 받아 딕셔너리 형태로 반환합니다.
    """
    try:
        성별 = int(input("성별 (남성: 1, 여성: 0): "))
        연령대코드 = int(input("연령대코드 (예: 7): "))
        신장 = float(input("신장(cm): "))
        체중 = float(input("체중(kg): "))
        수축기_혈압 = float(input("수축기 혈압(mmHg): "))
        이완기_혈압 = float(input("이완기 혈압(mmHg): "))

        # 입력 데이터를 딕셔너리로 반환
        return {
            '성별': 성별,
            '연령대코드': 연령대코드,
            '신장': 신장,
            '체중': 체중,
            '수축기 혈압': 수축기_혈압,
            '이완기 혈압': 이완기_혈압
        }
    except ValueError as e:
        print("잘못된 입력입니다. 숫자로 입력하세요.")
        return None
'''
# 사용 예제
if __name__ == "__main__":
    # 모델 파일 경로
    model_file_path = "/content/drive/MyDrive/석사 /2기/헬스케어데이터사이언스/헬스케어데이터사이언스/서민주/LogisticRegression_model.joblib"

    # 모델 로드
    model = load_model(model_file_path)
    #print("질병 예측을 위한 데이터를 입력하세요.")
    #single_input = get_user_input()

    # 한 명의 샘플 입력 데이터
    single_input = {
        '성별': 1,
        '연령대코드': 7,
        '신장': 175,
        '체중': 70,
        '수축기 혈압': 120,
        '이완기 혈압': 80
    }

    # 질병 확률 예측 및 속도 측정
    max_class, max_prob, inference_time = predict_disease_for_one(model, single_input)
    print("=======입력=======")
    print(single_input)
    print("-------출력-------")
    print(f"가장 높은 확률 클래스: {max_class},확률: {max_prob}")
    print(f"모델 추론 시간: {inference_time:.6f} 초")



=======입력=======
{'성별': 1, '연령대코드': 7, '신장': 175, '체중': 70, '수축기 혈압': 120, '이완기 혈압': 80}
-------출력-------
가장 높은 확률 클래스: 1,확률: 0.8708398146384673
모델 추론 시간: 0.003055 초


#당뇨


In [14]:
# 독립변수 및 종속변수 정의
X = health_data[['성별', '연령대코드', '신장', '체중',
                 '수축기 혈압', '이완기 혈압']]
y = health_data['당뇨병']

In [15]:
# 클래스 불균형 해결 (언더샘플링)
# y=1인 데이터 수와 동일한 수로 y=0을 샘플링
X_minority = X[y == 1]
y_minority = y[y == 1]

X_majority = X[y == 0]
y_majority = y[y == 0]

# y=0인 데이터에서 무작위로 샘플링
X_majority_downsampled, y_majority_downsampled = resample(
    X_majority, y_majority,
    n_samples=len(y_minority),    # y=1의 수에 맞게 샘플링
    random_state=42,              # 재현성을 위한 시드 값
    replace=False                 # 중복 샘플링 안 함
)

# 샘플링한 데이터 결합
X_resampled = pd.concat([X_minority, X_majority_downsampled])
y_resampled = pd.concat([y_minority, y_majority_downsampled])

In [16]:
# 1. 데이터 준비 및 전처리
# SMOTE로 데이터 증강
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)

# CNN은 3D 텐서를 입력으로 받으므로 차원 추가
X_resampled = X_resampled.values.reshape(X_resampled.shape[0], X_resampled.shape[1], 1)  # (샘플 수, 특성 수, 채널 수)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)  # 스케일링 후 복원
X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

In [17]:
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'),  # 필터 크기 조정 및 SAME 패딩
    MaxPooling1D(pool_size=2),
    Dropout(0.4),# 0.3 최적
    Conv1D(filters=64, kernel_size=2, activation='relu', padding='same'),  # SAME 패딩
    MaxPooling1D(pool_size=2),
    Dropout(0.4),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

# 3. 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\Master\anaconda3\envs\uni\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_split=0.2, verbose=1)

y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.48).astype(int).reshape(-1)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

model.save("./diabetes_model.h5")
print("모델이 .h5 형식으로 저장되었습니다.")


Epoch 1/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.6118 - loss: 0.6551 - val_accuracy: 0.6577 - val_loss: 0.6212
Epoch 2/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - accuracy: 0.6463 - loss: 0.6304 - val_accuracy: 0.6511 - val_loss: 0.6199
Epoch 3/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - accuracy: 0.6455 - loss: 0.6273 - val_accuracy: 0.6622 - val_loss: 0.6143
Epoch 4/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.6530 - loss: 0.6255 - val_accuracy: 0.6564 - val_loss: 0.6211
Epoch 5/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.6493 - loss: 0.6274 - val_accuracy: 0.6599 - val_loss: 0.6182
Epoch 6/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 33s 5ms/step - accuracy: 0.6489 - loss: 0.6283 - val_accuracy: 0.6609 - val_loss: 0.6133
Epoch 7/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - accuracy: 0.6477 - loss: 0.6268 - val_accuracy: 0.6568 - val_loss: 0.6144
Epoch 8/20
6990/6990 ━━━━━━━━━━━━━━━━━━━━ 37s 5ms/step - accuracy: 0.6500 - loss: 0

Accuracy: 0.6666094420600859
              precision    recall  f1-score   support

           0       0.72      0.55      0.62      8738
           1       0.64      0.78      0.70      8737

    accuracy                           0.67     17475
   macro avg       0.68      0.67      0.66     17475
weighted avg       0.68      0.67      0.66     17475

모델이 .h5 형식으로 저장되었습니다.


acc 63, recall 93-> 모델 예측할 때 무조건 1

그래서 recall 좀 줄어도 확률 보여줄 수 있는 모델 사용

In [ ]:
# 1. 모델 로드
def load_model(file_path):
    model = tf.keras.models.load_model(file_path)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 2. 예측 함수
def predict_disease_for_one(model, input_data, framework="tensorflow"):
    # 입력 데이터를 DataFrame으로 변환
    input_df = pd.DataFrame([input_data])

    # 입력 데이터 검증
    required_columns = ['성별', '연령대코드', '신장', '체중', '수축기 혈압', '이완기 혈압']
    if not all(col in input_df.columns for col in required_columns):
        raise ValueError(f"Input data must contain columns: {required_columns}")

    # 모델 입력 형식에 맞게 변환
    input_array = input_df.values.reshape(1, -1)  # (1, feature_count)

    # 추론 시작 시간
    start_time = time.time()

    # 모델 예측
    probabilities = model.predict(input_array)
    if probabilities.shape[1] == 1:  # Binary classification
        class_1_prob = probabilities[0][0]
        class_0_prob = 1 - class_1_prob
        probabilities = [[class_0_prob, class_1_prob]]
        classes = [0, 1]
    else:  # Multi-class classification
        classes = list(range(probabilities.shape[1]))

    # 추론 종료 시간
    inference_time = time.time() - start_time

    # 가장 높은 확률과 해당 클래스 추출
    max_prob_idx = probabilities[0].index(max(probabilities[0]))  # 확률이 가장 높은 클래스 인덱스
    max_prob = probabilities[0][max_prob_idx]                    # 해당 확률
    max_class = classes[max_prob_idx]                            # 해당 클래스

    return max_class, max_prob, inference_time

# 사용 예제
if __name__ == "__main__":
    # 모델 파일 경로 (.h5 파일)
    model_file_path = "/content/drive/MyDrive/석사 /2기/헬스케어데이터사이언스/헬스케어데이터사이언스/서민주/acc66_recall84.h5"

    # 모델 로드
    model = load_model(model_file_path)

    # 한 명의 샘플 입력 데이터
    single_input = {
        '성별': 1,
        '연령대코드':7,
        '신장': 175,
        '체중': 70,
        '수축기 혈압': 130,
        '이완기 혈압': 80
    }

    # 질병 확률 예측 및 속도 측정
    tf_max_class, tf_max_prob, tf_inference_time = predict_disease_for_one(model, single_input, framework="tensorflow")
    print("======= 입력 =======")
    print(f"입력 데이터: {single_input}")
    print("------- 출력 -------")

    print(f"가장 높은 확률 클래스: {tf_max_class}, 확률: {tf_max_prob}")
    print(f"모델 추론 시간: {tf_inference_time:.6f} 초")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
======= 입력 =======
입력 데이터: {'성별': 1, '연령대코드': 7, '신장': 175, '체중': 70, '수축기 혈압': 130, '이완기 혈압': 80}
------- 출력 -------
가장 높은 확률 클래스: 1, 확률: 0.9999997615814209
모델 추론 시간: 0.296774 초
